In [1]:
%matplotlib inline

In [2]:
import numpy as np
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from skimage.feature import blob_dog, blob_log, blob_doh
from math import sqrt
import cv2 as cv
from typing import Tuple
from tqdm.notebook import tqdm
from os import makedirs
from os.path import basename, splitext
import glob
from os.path import join

In [3]:
def get_filename(fp: str, prune_ext: bool = True) -> str:
    """Get the filename of a filepath.

    Args:
        fp: Filepath.
        prune_ext: Remove extension.
    
    Returns:
        Filename.

    """
    fn = basename(fp)

    if prune_ext:
        fn = splitext(fn)[0]

    return fn

def imread(fp: str, fmt: str = 'rgb', grayscale: bool = False) -> np.ndarray:
    """
    Read image file*.

    * Only supports RGB images currently, in the future we will look add 
    support for RGBA and grayscale images.
    
    Args:
        fp (str): Filepath to image.
        fmt (str): Format to read image as: 'rgb', 'bgr', 'gray'.
        grayscale (bool): Will be deprecated in the future, similar behavior can
            be achieved by setting format to 'gray'. Read image as grayscale.
    
    Returns:
        (numpy.ndarray) Image as numpy array.
    
    """
    assert fmt in ('rgb', 'bgr', 'gray'), "fmt must be 'rgb', 'bgr' or 'gray'."
    
    if grayscale:
        return cv.imread(fp, cv.IMREAD_GRAYSCALE)
    
    img = cv.imread(fp)
    
    if fmt == 'rgb':
        return cv.cvtColor(img, cv.COLOR_BGR2RGB)
    elif fmt == 'gray':
        return cv.cvtColor(img, cv.IMREAD_GRAYSCALE)
    else:
        return img

def blob_detect(fp: str, kwargs: dict, r_thr: int = 5, plot: bool = False,
                figsize: Tuple[int, int] = (7, 7), save_dir: str = None) -> str:
    """Detect blobs in an image, .
    
    Args:
        fp: Filepath of image.
        kwargs: Key-word arguments passed to skimage.feature.blob_log.
        r_thr: Remove blobs with radii smaller than this value.
        plot: Plot figures if True.
        figsize: Size of figures to plot.
        save_dir: Directory to save label text files.
    
    Returns:
        The blob coordinates in string format.
    
    """
    img = imread(fp, grayscale=True)        
    h, w = img.shape[:2]
    
    if plot:
        # Draw on the image.
        plt.figure(figsize=figsize)
        plt.imshow(img, cmap='gray')
        plt.title('Image', fontsize=16)
        plt.show()
    
    print(f'Size of image: {w} x {h}.')
    blobs = blob_log(img, **kwargs)
    
    # Add radious in their column.
    blobs[:, 2] = blobs[:, 2] * sqrt(2)
    
    print(f'{len(blobs)} number of blobs detected.')
    
    # Filter the blobs: 
    if plot:   
        img = cv.cvtColor(img, cv.COLOR_GRAY2RGB)
    lines = ''
    
    # Filter blobs.
    blobs = [blob for blob in blobs if blob[2] > r_thr]
    
    print(f'{len(blobs)} number of blobs after radii filtering.')

    for blob in blobs:
        y, x, r = blob.astype(int)
        
        x1, y1 = x - r, y - r
        x2, y2 = x + r, y + r
        
        lines += f'0 {x / w:4f} {y / h:4f} {(x2-x1) / w:4f} {(y2-y1) / h:4f}\n'
        
        if plot:
            img = cv.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
        
    if save_dir is not None:
        # Save the file.
        with open(join(save_dir, f'{get_filename(fp)}.txt'), 'w') as fh:
            fh.write(lines.strip())
        
    if plot:
        plt.figure(figsize=figsize)
        plt.imshow(img, cmap='gray')
        plt.title('Blobs', fontsize=16)
        plt.show()
        
    return lines

In [10]:
src_dir = './datasets/yolov8/images'
img_fps = sorted([fp for fp in glob.glob(join(src_dir, '*.png'))])
src_dir_labels = './datasets/yolov8'
if not img_fps:
    print("No PNG files found in the specified directory.")
else:
    label_dir = join(src_dir_labels, 'labels')
    makedirs(label_dir, exist_ok=True)
    print(f"Found {len(img_fps)} PNG files.")
label_dir = join(src_dir_labels, 'labels')
makedirs(label_dir, exist_ok=True)

Found 8 PNG files.


In [11]:
kwargs = {
    'max_sigma': 30, 
    'num_sigma': 15, 
    'threshold': 0.05
}

for fp in tqdm(img_fps):
    _ = blob_detect(fp, kwargs=kwargs, save_dir=label_dir, plot=False)

  0%|          | 0/8 [00:00<?, ?it/s]

Size of image: 3711 x 3735.
55410 number of blobs detected.
1477 number of blobs after radii filtering.
Size of image: 4186 x 4212.
36301 number of blobs detected.
1680 number of blobs after radii filtering.
Size of image: 2894 x 2366.
8161 number of blobs detected.
534 number of blobs after radii filtering.
Size of image: 3251 x 3718.
60905 number of blobs detected.
2032 number of blobs after radii filtering.
Size of image: 3225 x 3282.
4254 number of blobs detected.
1139 number of blobs after radii filtering.
Size of image: 3241 x 2791.
4964 number of blobs detected.
779 number of blobs after radii filtering.
Size of image: 3695 x 2813.
8039 number of blobs detected.
1117 number of blobs after radii filtering.
Size of image: 2309 x 1896.
2475 number of blobs detected.
321 number of blobs after radii filtering.


In [4]:
from utils import read_yolo_label, im_to_txt_path, corners_to_polygon, tile_roi_with_labels, tile_roi_with_labels_wrapper

In [5]:
from girder_client import GirderClient
from multiprocessing import Pool
from typing import List, Union, Tuple
from pandas import DataFrame, concat
from typing import Tuple
from geopandas import GeoDataFrame
from os import makedirs
from os.path import isfile, join
from typing import Union, Tuple
import numpy as np
from os.path import basename, splitext
import torch

def imwrite(fp: str, img: np.ndarray, grayscale: bool = False):
    """Write image to file.
    
    Args:
        fp: Filepath to save image.
        img: Image to save.
        grayscale: True to save image as a grayscale image, otherwise it is
            saved as an RGB image.
    
    """
    if grayscale:
        cv.imwrite(fp, img)
    else:
        cv.imwrite(fp, cv.cvtColor(img, cv.COLOR_RGB2BGR))

def imread(fp: str, fmt: str = 'rgb', grayscale: bool = False) -> np.ndarray:
    """
    Read image file*.

    * Only supports RGB images currently, in the future we will look add 
    support for RGBA and grayscale images.
    
    Args:
        fp (str): Filepath to image.
        fmt (str): Format to read image as: 'rgb', 'bgr', 'gray'.
        grayscale (bool): Will be deprecated in the future, similar behavior can
            be achieved by setting format to 'gray'. Read image as grayscale.
    
    Returns:
        (numpy.ndarray) Image as numpy array.
    
    """
    assert fmt in ('rgb', 'bgr', 'gray'), "fmt must be 'rgb', 'bgr' or 'gray'."
    
    if grayscale:
        return cv.imread(fp, cv.IMREAD_GRAYSCALE)
    
    img = cv.imread(fp)
    
    if fmt == 'rgb':
        return cv.cvtColor(img, cv.COLOR_BGR2RGB)
    elif fmt == 'gray':
        return cv.cvtColor(img, cv.IMREAD_GRAYSCALE)
    else:
        return img
        

In [6]:
fps = glob.glob("/code/ml-tissue-detection/datasets/yolov8/images/*.png")

SAVE_DIR= '/code/ml-tissue-detection/datasets/yolov8'
tiles_dir = join(SAVE_DIR, 'tiles')

tiles_df = tile_roi_with_labels_wrapper(
    fps,
    tiles_dir,
    tile_size=1280,
    stride=960,
    fill=0,
    notebook=True,
    grayscale=False
)

tiles_df.head()

  0%|          | 0/8 [00:00<?, ?it/s]

,fp,roi_fp,x,y,tile_size
0,/code/ml-tissue-detection/datasets/yolov8/tile...,/code/ml-tissue-detection/datasets/yolov8/imag...,0,0,1280
1,/code/ml-tissue-detection/datasets/yolov8/tile...,/code/ml-tissue-detection/datasets/yolov8/imag...,0,960,1280
2,/code/ml-tissue-detection/datasets/yolov8/tile...,/code/ml-tissue-detection/datasets/yolov8/imag...,0,1920,1280
3,/code/ml-tissue-detection/datasets/yolov8/tile...,/code/ml-tissue-detection/datasets/yolov8/imag...,0,2880,1280
4,/code/ml-tissue-detection/datasets/yolov8/tile...,/code/ml-tissue-detection/datasets/yolov8/imag...,960,0,1280


In [7]:
from sklearn.model_selection import train_test_split
import yaml
fps = sorted(fps)
train_fps, val_fps = train_test_split(fps, train_size=0.8)

val_txt_fp = join(SAVE_DIR, 'val.txt')

with open(join(SAVE_DIR, 'dataset.yaml'), 'w') as fh:
    yaml.safe_dump(
        {'nc': 1, 'names': ['nuclei'], 'path': SAVE_DIR, 'train': 'train.txt',
         'val': 'val.txt'}, 
        fh
    )
    
with open(join(SAVE_DIR, 'train.txt'), 'w') as fh:
    fh.write(
        '\n'.join(tiles_df[tiles_df.roi_fp.isin(train_fps)].fp.tolist()).strip()
    )
    
with open(join(SAVE_DIR, 'val.txt'), 'w') as fh:
    fh.write(
        '\n'.join(tiles_df[tiles_df.roi_fp.isin(val_fps)].fp.tolist()).strip()
    )

In [8]:
!pip install ultralytics

In [9]:
from ultralytics import YOLO

In [10]:
model = YOLO('yolov8n.yaml').load('yolov8n.pt')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [11]:
results = model.train(data='./datasets/yolov8/dataset.yaml', epochs=50, imgsz=640)

New https://pypi.org/project/ultralytics/8.0.215 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.211 🚀 Python-3.9.15 torch-2.1.0+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=./datasets/yolov8/dataset.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=Fals

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
train: Scanning /code/ml-tissue-detection/datasets/yolov8/tiles/labels... 87 images, 0 backgrounds, 0 corrupt: 100%|
train: New cache created: /code/ml-tissue-detection/datasets/yolov8/tiles/labels.cache
val: Scanning /code/ml-tissue-detection/datasets/yolov8/tiles/labels... 21 images, 0 backgrounds, 0 corrupt: 100%|██
val: New cache created: /code/ml-tissue-detection/datasets/yolov8/tiles/labels.cache
Plotting labels to runs/detect/train25/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train25
Starting training for 50 epochs...

lr/pg0,▁▂▂▃▄▄▅▅▆▇▇▇████▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▁▁
lr/pg1,▁▂▂▃▄▄▅▅▆▇▇▇████▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▁▁
lr/pg2,▁▂▂▃▄▄▅▅▆▇▇▇████▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▁▁
metrics/mAP50(B),▁▁▂▃▆▆▆▇▇▇▇▇▇▇▇██▇█████████████████▇████
metrics/mAP50-95(B),▁▁▂▂▄▅▅▅▆▅▆▆▆▆▆▆▇▆▇▇▇▇▇██▇██▇▇██▇▇▇▇▇▇██
metrics/precision(B),▁▁▂▃▄▄▄▅▅▅▅▇▇▇▇▇█▇█████████████████▇████
metrics/recall(B),▁▁▃▄▆▇▇▇▇▇▇▇▇▇▇██▇██████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▅▄▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁


In [12]:
metrics = model.val()

Ultralytics YOLOv8.0.211 🚀 Python-3.9.15 torch-2.1.0+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /code/ml-tissue-detection/datasets/yolov8/tiles/labels.cache... 21 images, 0 backgrounds, 0 corrupt: 1
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:3
                   all         21       3783      0.812      0.829      0.861      0.483
Speed: 0.2ms preprocess, 9.8ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/train252


In [13]:
fp = '/code/ml-tissue-detection/datasets/yolov8/tiles/images/'
model = YOLO('/code/ml-tissue-detection/runs/detect/train242/weights/best.pt')

results = model.predict(
    fp,
    save_txt=True, save_conf=True, save=True, hide_conf=True, hide_labels=True
)

WARNING ⚠️ 'hide_conf' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'show_conf' instead.
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.2' in the future. Please use 'show_labels' instead.

image 1/108 /code/ml-tissue-detection/datasets/yolov8/tiles/images/7-27-2023 E15-46 IGHM GFAP-x0y0.png: 640x640 178 nucleis, 14.4ms
image 2/108 /code/ml-tissue-detection/datasets/yolov8/tiles/images/7-27-2023 E15-46 IGHM GFAP-x0y1920.png: 640x640 113 nucleis, 12.0ms
image 3/108 /code/ml-tissue-detection/datasets/yolov8/tiles/images/7-27-2023 E15-46 IGHM GFAP-x0y2880.png: 640x640 57 nucleis, 11.9ms
image 4/108 /code/ml-tissue-detection/datasets/yolov8/tiles/images/7-27-2023 E15-46 IGHM GFAP-x0y960.png: 640x640 131 nucleis, 11.8ms
image 5/108 /code/ml-tissue-detection/datasets/yolov8/tiles/images/7-27-2023 E15-46 IGHM GFAP-x1920y0.png: 640x640 200 nucleis, 12.0ms
image 6/108 /code/ml-tissue-detection/datasets/yolov8/tiles/images/7-27-202